# Align and remove under/overexposed images

This notebook removes any under/overexposed frames from timelapse experiments and aligns the images. 

### The aligned images will be used later in the object localisation and tracking steps. 

The structure of this notebook is:

1. Load images using the octopuslite dask loader.
2. Find over/underexposed images by measuring each channel and frame for average pixel intensity.
3. Select a reference channel to center the alignment on
4. Register alignment and save out transformation tensor
5. (Optional) Apply transformation matrix to all channels and save out images
6. Check images using Napari
7. Function to iterate over many experiments, many positions

In [1]:
import os
import glob
import enum
import numpy as np
from pystackreg import StackReg
from skimage.io import imsave, imread
from tqdm.auto import tqdm
from octopuslite import DaskOctopusLiteLoader
from skimage import transform as tf

In [6]:
import pystackreg


In [8]:
print(pystackreg.__version__)

0.2.5


In [2]:
def image_generator(files, crop = None):
    """
    Generator function for iterative procesessing of image files
    """
    
    if crop is None:
        for filename in files:
            img = imread(filename)
            yield img
    else:
        for filename in files:
            img = imread(filename)
            img = crop_image(img, crop)
            yield img

## 1. Find images, organise and load using octopuslite

Define root directory and specific experiment and location to align

In [3]:
root_dir = '/home/nathan/data/kraken/ras'
expt = 'ND0012'
pos = 'Pos0'

Create new subdir for image files and move them all there. This is so that the miscelleanous non-image files (such as transformation matrices and tracking files) are easy to access later on and not lost amongst many single frame timelapse images.

In [4]:
image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
if not os.path.exists(image_path):
    os.mkdir(image_path)
    files = sorted(glob.glob(f'{root_dir}/{expt}/{pos}/*.tif'))
    for file in files:
        os.rename(file, file.replace(f'{pos}', f'{pos}/{pos}_images'))

Lazily load image array and associated information using dask octopuslite-loader and display channels found. Note the optional background removal is not invoked at this stage.

In [5]:
images = DaskOctopusLiteLoader(image_path, remove_background = False)
print([channel.name for channel in images.channels])

['BRIGHTFIELD', 'GFP', 'RFP', 'IRFP', 'MASK']


## 2. Identify under/overexposed images and display average channel brightness

In [ ]:
%%time
# pixel range criteria
max_pixel, min_pixel = 200, 2
# set empty dict arrays for mean values 
mean_arrays = {}
# set for dodgy frames (only unique entries)
dodgy_frame_list = set([])
#iterate over channels
for channel in tqdm(images.channels, desc = f'Finding mean values of image channels'):
    # find mean value of each frame in each channel
    mean_arrays[channel.name] = [np.mean(img) for img in image_generator(images.files(channel.name))]
    # iterate over frames
    for frame, mean_value in enumerate(mean_arrays[channel.name]):
        # check to see if mean frame pixel value meets criteria
        if max_pixel < mean_value or mean_value < min_pixel:
            # if so add to delete list
            dodgy_frame_list.add(frame)
# format delete list to only include single values
dodgy_frame_list = list(dodgy_frame_list)
print('Number of under/over-exposed frames:', len(dodgy_frame_list))

Finding mean values of image channels:   0%|          | 0/5 [00:00<?, ?it/s]

### 2a. Filter blanks from main image folder into separate directory

This step is optional as there is a parameter within `DaskOctopusLiteLoader` that filters the images, but employing that every time you load images is time consuming for large data sets

In [30]:
# check if blanks dir exists and make if not
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_blanks'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_blanks')
# move blank images into this directory
for channel in images.channels:
    for f in images.files(channel.name):
        for i in dodgy_frame_list:
            if str(i).zfill(9) in f:
                os.rename(f, f.replace('_images', '_blanks'))
# reload image arrays now that blanks filtered
images = DaskOctopusLiteLoader(image_path, remove_background = False)
images['gfp']

dask.array<stack, shape=(1638, 1352, 1688), dtype=uint8, chunksize=(1, 1352, 1688), chunktype=numpy.ndarray>

## 3. Select reference image to base alignment around

Display the average intensities of each channel. The automatically-measured brightest channel isn't necessarily the best.

In [31]:
print('Average channel brightness for selection of reference image:')
for channel in images.channels:
    print(f'{channel.value}: {channel.name}:', np.mean(mean_arrays[channel.name]))

Average channel brightness for selection of reference image:
0: BRIGHTFIELD: 40.96403863128827
1: GFP: 45.97970142886046
2: RFP: 6.969645431239206
3: IRFP: 47.732877102157886


In [33]:
# manually select reference channel by adding index
reference_channel = images.channels[1]
# automatically select reference channel from max average pixel value (ie. brightest channel)
#reference_channel = images.channels[max([(channel.value, np.mean(mean_arrays[channel.name])) for channel in images.channels])[0]]
reference_channel.name

'GFP'

#### 3a. Set cropped area of reference image to base alignment around 
Cropping as alignment struggles on large arrays such as `shape = (1200,1353,1682)`, this step is optional but you will still need to run `.compute()` on the dask array to load the image into memory to perform the alignment.

In [37]:
%%time
# crop central window out of reference image
reference_image = DaskOctopusLiteLoader(image_path, 
                                        crop = (500, 500)
                                       )[reference_channel.name].compute()
reference_image.shape

Using cropping: (500, 500)
CPU times: user 4min 53s, sys: 3min 39s, total: 8min 32s
Wall time: 55.6 s


(1638, 500, 500)

## 4. Register alignment and save out transformation tensor
Transformation tensor is a 3D series of transformation matrices over time

In [42]:
%%time
# create operator using transformation type (translation)
sr = StackReg(StackReg.TRANSLATION) 

# register each frame to the previous as transformation matrices/tensor
transform_tensor = sr.register_stack(reference_image, reference = 'previous')

# save out transform tensor
np.save(f'{root_dir}/{expt}/{pos}/{reference_channel.name.lower()}_transform_tensor.npy', transform_tensor)

/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


CPU times: user 3min 19s, sys: 3.78 s, total: 3min 23s
Wall time: 3min 23s


In [47]:
transform_tensor.shape

(1638, 3, 3)

## 5. (Optional) Apply transformation matrix to all channels and save out images in separate directory

Consumes a lot of time and space to replicate images with minor translational shifts, it is advised to just use the transform parameter in the `DaskOctopusLiteLoader`. 

In [14]:
%%time
### iterating over channels
# create aligned image dir if does not exist 
if not os.path.exists(f'{root_dir}/{expt}/{pos}/{pos}_aligned'):
    os.mkdir(f'{root_dir}/{expt}/{pos}/{pos}_aligned')
# iterate over channels
for channel in images.channels:
    #iterate over all images in channel
    for i in tqdm(range(len(transform_tensor)), 
                  desc = f'Aligning {channel.name.lower()} channel {channel.value+1}/{len(images.channels)}'):
        # load specific transform matrix for that frame
        transform_matrix = tf.EuclideanTransform(matrix = transform_tensor[i,...],
                                                 rotation = None)
        # transform image
        transformed_image = (tf.warp(images[channel.name][i,...].compute(), 
                                     transform_matrix, preserve_range=True)).astype(np.uint8)
        # set transformed image pathname by editing base dir
        fn = images.files(channel.name)[i].replace('_images', '_aligned')
        # save trans image out
        imsave(fn, transformed_image, check_contrast=False)

Aligning irfp channel 4/4: 100%|██████████| 1067/1067 [03:04<00:00,  5.79it/s]

CPU times: user 5min 24s, sys: 51.4 s, total: 6min 15s
Wall time: 11min 45s


## 6. Check alignment using Napari

In [53]:
import napari

In [56]:
aligned_images = DaskOctopusLiteLoader(image_path, 
                                       #crop = (1200,1600), 
                                       transforms = f'{root_dir}/{expt}/{pos}/gfp_transform_tensor.npy',
                                       remove_background=False)
viewer = napari.Viewer()
for channel in aligned_images.channels:
    viewer.add_image(aligned_images[channel.name], 
                     name = channel.name, 
                     blending = 'additive', 
                     contrast_limits = [0,255])

## Batch execute

Do all of the above but for many experiment IDs and many positions

In [ ]:
root_dir = '/home/nathan/data/kraken/ras'
expt_list = ['ND0010']
max_pixel = 200
min_pixel = 2
alignment_channel = 'gfp'
crop_area = 500
save_out_images = False ### this does not save out a copy of the images, only the transformation matrix
overwrite = False ### this checks for any prexisting transformations and does not overwrite

# trying reference to first image first

In [2]:
from multiprocessing import Pool
cpus = os.cpu_count()
cpus

12

In [2]:
from natsort import natsorted

In [3]:
expt = 'ND0010'
root_dir = '/home/nathan/data/kraken/ras'
pos_list = natsorted([pos for pos in os.listdir(f'{root_dir}/{expt}') 
               if 'Pos' in pos 
               and os.path.isdir(f'{root_dir}/{expt}/{pos}')]  )


In [5]:
def align(pos):
    
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    crop_area = 500
    # crop central window out of reference image with blanks removed
    reference_image = DaskOctopusLiteLoader(image_path, 
                                            crop = (crop_area, crop_area), 
                                            transforms=None
                                           )['gfp'].compute() 
    ## check that image is sequentially correct
    
    ### Register alignment
    print('Registering alignment for', pos, expt)
    # create operator using transformation type (translation)
    sr = StackReg(StackReg.TRANSLATION) 
    # register each frame to the previous as transformation matrices/tensor
    transform_tensor = sr.register_stack(reference_image, reference = 'first')
    # save out transform tensor
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_2.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')
    return


In [6]:
def show_progress(current_iteration, end_iteration):
    print(f"Registering {current_iteration} of {end_iteration} images")

def align_overnight(pos):
  
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    crop_area = 1300
    # crop central window out of reference image with blanks removed
    reference_image = DaskOctopusLiteLoader(image_path, 
                                            crop = (crop_area, crop_area), 
                                            transforms=None, 
                                           )['gfp'].compute() 
    ## check that image is sequentially correct
    
    ### Register alignment
    print('Registering alignment for', pos, expt)
    # create operator using transformation type (translation)
    sr = StackReg(StackReg.TRANSLATION) 
    # register each frame to the previous as transformation matrices/tensor
    transform_tensor = sr.register_stack(reference_image, reference = 'first', progress_callback=show_progress)
    # save out transform tensor
    np.save(f'{root_dir}/{expt}/{pos}/gfp_transform_tensor_5.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')
    return

In [13]:
for pos in pos_list:
    align_overnight(pos)

Starting ND0010/Pos0
Using cropping: (1300, 1300)
Registering alignment for Pos0 ND0010


/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


Registering 1 of 1189 images
Registering 2 of 1189 images
Registering 3 of 1189 images
Registering 4 of 1189 images
Registering 5 of 1189 images
Registering 6 of 1189 images
Registering 7 of 1189 images
Registering 8 of 1189 images
Registering 9 of 1189 images
Registering 10 of 1189 images
Registering 11 of 1189 images
Registering 12 of 1189 images
Registering 13 of 1189 images
Registering 14 of 1189 images
Registering 15 of 1189 images
Registering 16 of 1189 images
Registering 17 of 1189 images
Registering 18 of 1189 images
Registering 19 of 1189 images
Registering 20 of 1189 images
Registering 21 of 1189 images
Registering 22 of 1189 images
Registering 23 of 1189 images
Registering 24 of 1189 images
Registering 25 of 1189 images
Registering 26 of 1189 images
Registering 27 of 1189 images
Registering 28 of 1189 images
Registering 29 of 1189 images
Registering 30 of 1189 images
Registering 31 of 1189 images
Registering 32 of 1189 images
Registering 33 of 1189 images
Registering 34 of 1

Registering 269 of 1189 images
Registering 270 of 1189 images
Registering 271 of 1189 images
Registering 272 of 1189 images
Registering 273 of 1189 images
Registering 274 of 1189 images
Registering 275 of 1189 images
Registering 276 of 1189 images
Registering 277 of 1189 images
Registering 278 of 1189 images
Registering 279 of 1189 images
Registering 280 of 1189 images
Registering 281 of 1189 images
Registering 282 of 1189 images
Registering 283 of 1189 images
Registering 284 of 1189 images
Registering 285 of 1189 images
Registering 286 of 1189 images
Registering 287 of 1189 images
Registering 288 of 1189 images
Registering 289 of 1189 images
Registering 290 of 1189 images
Registering 291 of 1189 images
Registering 292 of 1189 images
Registering 293 of 1189 images
Registering 294 of 1189 images
Registering 295 of 1189 images
Registering 296 of 1189 images
Registering 297 of 1189 images
Registering 298 of 1189 images
Registering 299 of 1189 images
Registering 300 of 1189 images
Register

Registering 534 of 1189 images
Registering 535 of 1189 images
Registering 536 of 1189 images
Registering 537 of 1189 images
Registering 538 of 1189 images
Registering 539 of 1189 images
Registering 540 of 1189 images
Registering 541 of 1189 images
Registering 542 of 1189 images
Registering 543 of 1189 images
Registering 544 of 1189 images
Registering 545 of 1189 images
Registering 546 of 1189 images
Registering 547 of 1189 images
Registering 548 of 1189 images
Registering 549 of 1189 images
Registering 550 of 1189 images
Registering 551 of 1189 images
Registering 552 of 1189 images
Registering 553 of 1189 images
Registering 554 of 1189 images
Registering 555 of 1189 images
Registering 556 of 1189 images
Registering 557 of 1189 images
Registering 558 of 1189 images
Registering 559 of 1189 images
Registering 560 of 1189 images
Registering 561 of 1189 images
Registering 562 of 1189 images
Registering 563 of 1189 images
Registering 564 of 1189 images
Registering 565 of 1189 images
Register

...

Registering 1153 of 1193 images
Registering 1154 of 1193 images
Registering 1155 of 1193 images
Registering 1156 of 1193 images
Registering 1157 of 1193 images
Registering 1158 of 1193 images
Registering 1159 of 1193 images
Registering 1160 of 1193 images
Registering 1161 of 1193 images
Registering 1162 of 1193 images
Registering 1163 of 1193 images
Registering 1164 of 1193 images
Registering 1165 of 1193 images
Registering 1166 of 1193 images
Registering 1167 of 1193 images
Registering 1168 of 1193 images
Registering 1169 of 1193 images
Registering 1170 of 1193 images
Registering 1171 of 1193 images
Registering 1172 of 1193 images
Registering 1173 of 1193 images
Registering 1174 of 1193 images
Registering 1175 of 1193 images
Registering 1176 of 1193 images
Registering 1177 of 1193 images
Registering 1178 of 1193 images
Registering 1179 of 1193 images
Registering 1180 of 1193 images
Registering 1181 of 1193 images
Registering 1182 of 1193 images
Registering 1183 of 1193 images
Register

Registering 223 of 1190 images
Registering 224 of 1190 images
Registering 225 of 1190 images
Registering 226 of 1190 images
Registering 227 of 1190 images
Registering 228 of 1190 images
Registering 229 of 1190 images
Registering 230 of 1190 images
Registering 231 of 1190 images
Registering 232 of 1190 images
Registering 233 of 1190 images
Registering 234 of 1190 images
Registering 235 of 1190 images
Registering 236 of 1190 images
Registering 237 of 1190 images
Registering 238 of 1190 images
Registering 239 of 1190 images
Registering 240 of 1190 images
Registering 241 of 1190 images
Registering 242 of 1190 images
Registering 243 of 1190 images
Registering 244 of 1190 images
Registering 245 of 1190 images
Registering 246 of 1190 images
Registering 247 of 1190 images
Registering 248 of 1190 images
Registering 249 of 1190 images
Registering 250 of 1190 images
Registering 251 of 1190 images
Registering 252 of 1190 images
Registering 253 of 1190 images
Registering 254 of 1190 images
Register

Registering 488 of 1190 images
Registering 489 of 1190 images
Registering 490 of 1190 images
Registering 491 of 1190 images
Registering 492 of 1190 images
Registering 493 of 1190 images
Registering 494 of 1190 images
Registering 495 of 1190 images
Registering 496 of 1190 images
Registering 497 of 1190 images
Registering 498 of 1190 images
Registering 499 of 1190 images
Registering 500 of 1190 images
Registering 501 of 1190 images
Registering 502 of 1190 images
Registering 503 of 1190 images
Registering 504 of 1190 images
Registering 505 of 1190 images
Registering 506 of 1190 images
Registering 507 of 1190 images
Registering 508 of 1190 images
Registering 509 of 1190 images
Registering 510 of 1190 images
Registering 511 of 1190 images
Registering 512 of 1190 images
Registering 513 of 1190 images
Registering 514 of 1190 images
Registering 515 of 1190 images
Registering 516 of 1190 images
Registering 517 of 1190 images
Registering 518 of 1190 images
Registering 519 of 1190 images
Register

Registering 753 of 1190 images
Registering 754 of 1190 images
Registering 755 of 1190 images
Registering 756 of 1190 images
Registering 757 of 1190 images
Registering 758 of 1190 images
Registering 759 of 1190 images
Registering 760 of 1190 images
Registering 761 of 1190 images
Registering 762 of 1190 images
Registering 763 of 1190 images
Registering 764 of 1190 images
Registering 765 of 1190 images
Registering 766 of 1190 images
Registering 767 of 1190 images
Registering 768 of 1190 images
Registering 769 of 1190 images
Registering 770 of 1190 images
Registering 771 of 1190 images
Registering 772 of 1190 images
Registering 773 of 1190 images
Registering 774 of 1190 images
Registering 775 of 1190 images
Registering 776 of 1190 images
Registering 777 of 1190 images
Registering 778 of 1190 images
Registering 779 of 1190 images
Registering 780 of 1190 images
Registering 781 of 1190 images
Registering 782 of 1190 images
Registering 783 of 1190 images
Registering 784 of 1190 images
Register

Registering 1017 of 1190 images
Registering 1018 of 1190 images
Registering 1019 of 1190 images
Registering 1020 of 1190 images
Registering 1021 of 1190 images
Registering 1022 of 1190 images
Registering 1023 of 1190 images
Registering 1024 of 1190 images
Registering 1025 of 1190 images
Registering 1026 of 1190 images
Registering 1027 of 1190 images
Registering 1028 of 1190 images
Registering 1029 of 1190 images
Registering 1030 of 1190 images
Registering 1031 of 1190 images
Registering 1032 of 1190 images
Registering 1033 of 1190 images
Registering 1034 of 1190 images
Registering 1035 of 1190 images
Registering 1036 of 1190 images
Registering 1037 of 1190 images
Registering 1038 of 1190 images
Registering 1039 of 1190 images
Registering 1040 of 1190 images
Registering 1041 of 1190 images
Registering 1042 of 1190 images
Registering 1043 of 1190 images
Registering 1044 of 1190 images
Registering 1045 of 1190 images
Registering 1046 of 1190 images
Registering 1047 of 1190 images
Register

In [7]:
for pos in tqdm(pos_list):
   
    #align_overnight(pos)
    expt = 'ND0010'
    root_dir = '/home/nathan/data/kraken/ras'
    
    print(f'Starting {expt}/{pos}')

    ### create new subdir of for raw files and move them all there
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    
    ### pre load files from raw file dir 
    images = DaskOctopusLiteLoader(image_path, remove_background= False)
    
    crop_area = 500
    
    N_files = len(images['brightfield'])
    transform_tensor_sum = []
    for chunk in tqdm(range(0,N_files,int(N_files/10))):
    # crop central window out of reference image with blanks removed
        reference_image = DaskOctopusLiteLoader(image_path, 
                                                #crop = (crop_area, crop_area), 
                                              # transforms='', 
                                               )['brightfield'][chunk:chunk+int(N_files/10)].compute() 
        ### Register alignment
        print('Registering alignment for', pos, chunk,chunk+int(N_files/10), 'frames')
        # create operator using transformation type (translation)# if self._transformer is not None
        sr = StackReg(StackReg.TRANSLATION) 
        # register each frame to the previous as transformation matrices/tensor
        transform_tensor = sr.register_stack(reference_image, reference = 'previous', progress_callback=show_progress)
        transform_tensor_sum.append(transform_tensor)
    #concatenate different chunks of trans tensor
    transform_tensor = np.concatenate(transform_tensor_sum, axis=0)
    #reshape
    #transform_tensor = np.reshape(transform_tensor, (len(transform_tensor)*len(transform_tensor[0]), 3, 3))
    # save out transform tensor
    np.save(f'{root_dir}/{expt}/{pos}/bf_transform_tensor_12.npy', transform_tensor)
    print(f'/{expt}/{pos}/ saved')

  0%|          | 0/12 [00:00<?, ?it/s]

Starting ND0010/Pos0


  0%|          | 0/10 [00:00<?, ?it/s]

Registering alignment for Pos0 0 119 frames


/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/site-packages/pystackreg/pystackreg.py:379: UserWarning: Detected axis 2 as the possible time axis for the stack due to its low variability, but axis 0 was supplied for registration. Are you sure you supplied the correct axis?
  warnings.warn(


...

Registering alignment for Pos11 0 119 frames
Registering 1 of 118 images
Registering 2 of 118 images
Registering 3 of 118 images
Registering 4 of 118 images
Registering 5 of 118 images
Registering 6 of 118 images
Registering 7 of 118 images
Registering 8 of 118 images
Registering 9 of 118 images
Registering 10 of 118 images
Registering 11 of 118 images
Registering 12 of 118 images
Registering 13 of 118 images
Registering 14 of 118 images
Registering 15 of 118 images
Registering 16 of 118 images
Registering 17 of 118 images
Registering 18 of 118 images
Registering 19 of 118 images
Registering 20 of 118 images
Registering 21 of 118 images
Registering 22 of 118 images
Registering 23 of 118 images
Registering 24 of 118 images
Registering 25 of 118 images
Registering 26 of 118 images
Registering 27 of 118 images
Registering 28 of 118 images
Registering 29 of 118 images
Registering 30 of 118 images
Registering 31 of 118 images
Registering 32 of 118 images
Registering 33 of 118 images
Registe

Registering 43 of 118 images
Registering 44 of 118 images
Registering 45 of 118 images
Registering 46 of 118 images
Registering 47 of 118 images
Registering 48 of 118 images
Registering 49 of 118 images
Registering 50 of 118 images
Registering 51 of 118 images
Registering 52 of 118 images
Registering 53 of 118 images
Registering 54 of 118 images
Registering 55 of 118 images
Registering 56 of 118 images
Registering 57 of 118 images
Registering 58 of 118 images
Registering 59 of 118 images
Registering 60 of 118 images
Registering 61 of 118 images
Registering 62 of 118 images
Registering 63 of 118 images
Registering 64 of 118 images
Registering 65 of 118 images
Registering 66 of 118 images
Registering 67 of 118 images
Registering 68 of 118 images
Registering 69 of 118 images
Registering 70 of 118 images
Registering 71 of 118 images
Registering 72 of 118 images
Registering 73 of 118 images
Registering 74 of 118 images
Registering 75 of 118 images
Registering 76 of 118 images
Registering 77

Registering 86 of 118 images
Registering 87 of 118 images
Registering 88 of 118 images
Registering 89 of 118 images
Registering 90 of 118 images
Registering 91 of 118 images
Registering 92 of 118 images
Registering 93 of 118 images
Registering 94 of 118 images
Registering 95 of 118 images
Registering 96 of 118 images
Registering 97 of 118 images
Registering 98 of 118 images
Registering 99 of 118 images
Registering 100 of 118 images
Registering 101 of 118 images
Registering 102 of 118 images
Registering 103 of 118 images
Registering 104 of 118 images
Registering 105 of 118 images
Registering 106 of 118 images
Registering 107 of 118 images
Registering 108 of 118 images
Registering 109 of 118 images
Registering 110 of 118 images
Registering 111 of 118 images
Registering 112 of 118 images
Registering 113 of 118 images
Registering 114 of 118 images
Registering 115 of 118 images
Registering 116 of 118 images
Registering 117 of 118 images
Registering 118 of 118 images
Registering alignment fo

Registering 9 of 118 images
Registering 10 of 118 images
Registering 11 of 118 images
Registering 12 of 118 images
Registering 13 of 118 images
Registering 14 of 118 images
Registering 15 of 118 images
Registering 16 of 118 images
Registering 17 of 118 images
Registering 18 of 118 images
Registering 19 of 118 images
Registering 20 of 118 images
Registering 21 of 118 images
Registering 22 of 118 images
Registering 23 of 118 images
Registering 24 of 118 images
Registering 25 of 118 images
Registering 26 of 118 images
Registering 27 of 118 images
Registering 28 of 118 images
Registering 29 of 118 images
Registering 30 of 118 images
Registering 31 of 118 images
Registering 32 of 118 images
Registering 33 of 118 images
Registering 34 of 118 images
Registering 35 of 118 images
Registering 36 of 118 images
Registering 37 of 118 images
Registering 38 of 118 images
Registering 39 of 118 images
Registering 40 of 118 images
Registering 41 of 118 images
Registering 42 of 118 images
Registering 43 

Registering 52 of 118 images
Registering 53 of 118 images
Registering 54 of 118 images
Registering 55 of 118 images
Registering 56 of 118 images
Registering 57 of 118 images
Registering 58 of 118 images
Registering 59 of 118 images
Registering 60 of 118 images
Registering 61 of 118 images
Registering 62 of 118 images
Registering 63 of 118 images
Registering 64 of 118 images
Registering 65 of 118 images
Registering 66 of 118 images
Registering 67 of 118 images
Registering 68 of 118 images
Registering 69 of 118 images
Registering 70 of 118 images
Registering 71 of 118 images
Registering 72 of 118 images
Registering 73 of 118 images
Registering 74 of 118 images
Registering 75 of 118 images
Registering 76 of 118 images
Registering 77 of 118 images
Registering 78 of 118 images
Registering 79 of 118 images
Registering 80 of 118 images
Registering 81 of 118 images
Registering 82 of 118 images
Registering 83 of 118 images
Registering 84 of 118 images
Registering 85 of 118 images
Registering 86